In [ ]:
import os

from rich import print as rprint

from entitysdk import Client, ProjectContext, models

entitycore_api_url = "http://127.0.0.1:8000"
project_context = ProjectContext(
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730",
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6",
)
token = os.getenv("ACCESS_TOKEN", "XXX")
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

## Register a Circuit

In [ ]:
species = client.search_entity(
    entity_type=models.Species, query={"name__ilike": "Mus musculus"}
).first()

subject = models.Subject(
    name="my-subject", description="my-subject-description", sex="male", species=species
)
subject = client.register_entity(subject)

brain_region = client.search_entity(entity_type=models.BrainRegion, query={"acronym": "CB"}).first()


circuit = models.Circuit(
    name="my-circuit",
    description="my-circuit",
    subject=subject,
    brain_region=brain_region,
    number_synapses=2,
    number_neurons=5,
    number_connections=10,
    scale="microcircuit",
    build_category="em_reconstruction",
)

circuit = client.register_entity(circuit)

## Register a SimulationCampaign

In [ ]:
campaign = models.SimulationCampaign(
    name="my-campaign",
    description="my-campaign-description",
    scan_parameters={"foo": "bar"},
    entity_id=circuit.id,
)
campaign = client.register_entity(campaign)

## Register Simulations

In [ ]:
for i in range(5):
    simulation = models.Simulation(
        name=f"sim-{i}",
        description=f"sim-{i}",
        scan_parameters={"foo": "bar"},
        entity_id=circuit.id,
        simulation_campaign_id=campaign.id,
    )
    client.register_entity(simulation)

## Fetch SimulationCampaign

In [ ]:
campaign = client.get_entity(entity_id=campaign.id, entity_type=models.SimulationCampaign)

In [ ]:
rprint(campaign.simulations)